In [1]:
from bs4 import BeautifulSoup
from bs4 import NavigableString
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from scipy.spatial.distance import pdist, squareform
import os

# Read in reference SVG

In [2]:
file = 'MOD2.subclass_classcol.constellation.svg'
with open(file) as fp:
    data = fp.read()
    soup = BeautifulSoup( data, 'xml' )

# Read in parsed information

In [13]:
input_directory = 'parsed'

file = os.path.join( input_directory, 'circles.csv')
circles = pd.read_csv( file )

file = os.path.join( input_directory, 'annotations.csv')
annotations = pd.read_csv( file )

file = os.path.join( input_directory, 'connections.csv')
connections = pd.read_csv( file )

file = os.path.join( input_directory, 'subclasses.csv')
subclasses = pd.read_csv( file )

file = os.path.join( input_directory, 'classes.csv')
classes = pd.read_csv( file )

In [4]:
subclasses.set_index('node', inplace=True )

# Create reorganized and labeled SVG

In [5]:
# create a new soup
newSoup = BeautifulSoup(features='xml')
newSvg = newSoup.new_tag('svg')
newSvg.attrs = soup.svg.attrs.copy()
newSoup.append(newSvg)

In [6]:
pocHost = 'http://35.92.115.7:8883'

In [7]:
output_file_name = 'test2.svg'
connections_layer = True
nodes_layer = True
nodes_subclass_color_layer = False
labels_layer = True

In [8]:
if connections_layer :
    # create connections group
    newGroup = newSoup.new_tag('g')
    newGroup['id'] = 'connections'
    newSvg.append(newGroup)
    
    for cindex, crow in connections.iterrows() :
    
        newPolygon = newSoup.new_tag('polygon')
        newPolygon.attrs['points'] = crow['points']
        newPolygon.attrs['fill'] = '#BBBBBB'
    
        nstart = subclasses.loc[crow['start_node'],'name']
        nend = subclasses.loc[crow['end_node'],'name']
        cstr = nstart + ':: ' + nend
        newTitle = newSoup.new_tag('title')
        newString = NavigableString( cstr )
        newTitle.append(newString)
        newPolygon.append( newTitle )
    
        newGroup.append(newPolygon)


In [9]:
if nodes_subclass_color_layer :
    # create nodes group
    newGroup = newSoup.new_tag('g')
    newGroup['id'] = 'nodes'
    newSvg.append(newGroup)
    
    for cindex, crow in circles.iterrows() :
    
        newLink = newSoup.new_tag('a')
        newLink.attrs['href'] = '%s/display_entity?entity_id=CS20230722_SUBC_%03d' % (pocHost,int(crow['node']))
        
        newPath = newSoup.new_tag('path')
        newPath.attrs['id'] = crow['id']
        newPath.attrs['d'] = crow['d']
        newPath.attrs['fill'] = subclasses.loc[crow['node'],'color_hex_triplet']
        
        newTitle = newSoup.new_tag('title')
        newString = NavigableString( subclasses.loc[crow['node'],'name'] )
        newTitle.append(newString)
        newPath.append(newTitle)
    
        newLink.append(newPath)
        newGroup.append(newLink)

In [10]:
if nodes_layer :
    # create nodes group
    newGroup = newSoup.new_tag('g')
    newGroup['id'] = 'nodes'
    newSvg.append(newGroup)
    
    for cindex, crow in circles.iterrows() :
    
        newLink = newSoup.new_tag('a')
        newLink.attrs['href'] = '%s/display_entity?entity_id=CS20230722_SUBC_%03d' % (pocHost,int(crow['node']))
        
        newPath = newSoup.new_tag('path')
        newPath.attrs['id'] = crow['id']
        newPath.attrs['d'] = crow['d']
        newPath.attrs['fill'] = subclasses.loc[crow['node'],'parent_color_hex_triplet']
        
        newTitle = newSoup.new_tag('title')
        newString = NavigableString( subclasses.loc[crow['node'],'name'] )
        newTitle.append(newString)
        newPath.append(newTitle)
    
        newLink.append(newPath)
        newGroup.append(newLink)

In [11]:
if labels_layer :
    # create label group
    newGroup = newSoup.new_tag('g')
    newGroup['id'] = 'labels'
    newSvg.append(newGroup)
    
    for cindex, crow in annotations.iterrows() :
    
        newLink = newSoup.new_tag('a')
        newLink.attrs['href'] = '%s/display_entity?entity_id=CS20230722_SUBC_%03d' % (pocHost,int(crow['node']))
        
        newText = newSoup.new_tag('text')
        newString = NavigableString( str(crow['node']) )
        newText.append(newString)
    
        newText.attrs['transform'] = crow['transform']
        newText.attrs['font-size'] = 7
        newText.attrs['font-family'] = "Arial"
        newText.attrs['fill'] = '#444444'
    
        newTitle = newSoup.new_tag('title')
        newString = NavigableString( subclasses.loc[crow['node'],'name'] )
        newTitle.append(newString)
        newText.append(newTitle)
    
        newLink.append(newText)
        newGroup.append(newLink)


In [12]:
output_directory = 'output'
os.makedirs( output_directory, exist_ok = True )
file = os.path.join( output_directory, output_file_name )
with open(file, "w") as f:
    f.write(str(newSoup))